[IPython Notebook](undo_redo.ipynb) |  [Python Script](undo_redo.py)

Advanced: Undo/Redo
============================

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new bundle.  See [Building a System](building_a_system.html) for more details.

In [1]:
%matplotlib inline

In [2]:
import phoebe2
from phoebe2 import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe2.logger(clevel='INFO')

b = phoebe2.Bundle.default_binary()

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


Enabling/Disabling Logging History
---------------------------------------------

Undo and redo support is built directly into the bundle.  Every time you make a change to a parameter or call a method, a new Parameter is created with the 'history' context.  These parameters then know how to undo or redo that action.  Of course, this can result in a large list of Parameters that you may not want - see the tutorial on [Settings](settings) for more details on how to change the log size or enable/disable history entirely.

By default history logging is off, so let's first enable it.

In [3]:
b.enable_history()

Undoing
---------------

First let's set a value so we know what we're expecting to undo

In [4]:
b['ra@system']

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

In [5]:
b['ra@system'] = 10
b['ra@system']

<Parameter: ra=10.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

The history context contains a list of parameters, all numbered and ugly.  But there is a convenience method which allows you to get history items by index - including reverse indexing.  This is probably the most common way to get a history item... and you'll most likely want the LATEST item.

In [6]:
b.get_history(-1)

<HistoryParameter: hist01 | keys: redo_func, redo_kwargs, undo_func, undo_kwargs>

In [7]:
print b.get_history(-1)['redo_func'], b.get_history(-1)['redo_kwargs']

set_value {'twig': 'ra@system', 'value': <Quantity 10.0 deg>}


Here you can see that redo_func and redo_kwargs shows exactly the last call we made to the bundle that actually changed something (we did b['ra@system'] = 10).  

We can also look at what will be called when we undo this item

In [8]:
print b.get_history(-1)['undo_func'], b.get_history(-1)['undo_kwargs']

set_value {'twig': 'ra@system', 'value': 0.0}


If we want, we can then automatically call that undo method (note that you can also pass the index to undo, but it does assume -1 by default)

In [9]:
b.undo()

And we can see that it did exactly what we expected.

In [10]:
b['ra@system']

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>